# Imports

In [2]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from collections import defaultdict

import common_functions
import numpy as np
import pandas as pd
import utils
from scipy import stats

# Aim of this notebook  
* Calculate the Estimated Daily Intake values based on the wristband and dust measurements



In [5]:
DUST_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath("dust.parquet.gzip")

SERUM_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath(
    "HBM4EU_E-waste_template_V3_all_data_INTERIM.parquet.gzip"
)

WRISTBAND_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath(
    "wristband.parquet.gzip"
)

dust=pd.read_parquet(DUST_DATA_PATH)
serum=pd.read_parquet(SERUM_DATA_PATH)
wristband=pd.read_parquet(WRISTBAND_DATA_PATH)